In [1]:
import pymongo
import requests
import pandas as pd

from os import getenv
from datetime import datetime
from dotenv import load_dotenv

from config import contract_collection, prices_collection
from helper import contract_list


load_dotenv()

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

In [4]:
from config import contract_collection, prices_collection
from helper import contract_list, get_collection_activity, get_collection_info



contract_db = contract_list()
print('Running...')

test = get_collection_activity(True, 20, community='artblocks')
new_bids = []
for item in test['activities']:
    if item['type'] == 'bid':
        if item['order']['criteria']['kind'] == 'collection':
            actions = {}
            actions['contract'] = item['contract']
            actions['collection_name'] = item['collection']['collectionName']
            actions['collection_id'] = item['order']['criteria']['data']['collection']['id']
            if item['order']['criteria']['data']['collection']['id'] not in contract_db:
                new_bids.append(actions)

if new_bids:   
    print(f'New Contracts: {len(new_bids)}')            
    contract_collection().insert_many(new_bids)

contract_db = contract_list()
prices_db = prices_collection()

new_bids = []
for c in contract_db:
    collections = get_collection_info(1, True, False, c_id=c)
    for collection in collections['collections']:
        project = {}
        project['Collection'] = collection['name']
        project['7day Volume'] = round(collection['volume']['7day'], 2)
        try:
            project['Lowest Floor Price'] = f"{str(collection['floorAsk']['sourceDomain'])} : {round(collection['floorAsk']['price']['amount']['decimal'], 3)} {collection['floorAsk']['price']['currency']['symbol']}"
            project['Highest Collection Offer'] = f"{str(collection['topBid']['sourceDomain'])} : {round(collection['topBid']['price']['amount']['decimal'], 3)} {collection['topBid']['price']['currency']['symbol']}"
            project['Spread'] = f"{round(((collection['floorAsk']['price']['amount']['decimal'] - collection['topBid']['price']['amount']['decimal']) / collection['floorAsk']['price']['amount']['decimal']) * 100, 2)}%"
        except TypeError:
            project['Lowest Floor Price'] = 'None'
            project['Highest Collection Offer'] = 'None'
            project['Spread'] = 'None'
        new_bids.append(project)
    
prices_db.delete_many({})
prices_db.insert_many(new_bids)


Running...


In [3]:
collection

{'id': '0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270:369000000:369999999',
 'slug': 'century-2052-by-casey-reas',
 'createdAt': '2022-10-20T22:56:15.451Z',
 'name': 'CENTURY 2052 by Casey REAS',
 'image': 'https://media.artblocks.io/369000000.png',
 'banner': None,
 'discordUrl': None,
 'externalUrl': 'https://reas.com',
 'twitterUsername': None,
 'openseaVerificationStatus': None,
 'description': "This artwork will “die” on 20 October 2052.\n\nToday I’m 50.\nWill I live to be 80?\nWill I outlive this artwork?\nWill this artwork outlive me?\n\nThe original CENTURY project is autobiographical. Creating art and experiencing art is the core of my life, and CENTURY is my homage to 20th Century artists. It’s a synthesis of their ideas.\n\nCENTURY 2052 brings the endeavor to the 21st Century. It’s released as 50 unique artworks, one for each year since I was born. The artwork transitions from life to death at 6:44pm EST on October 20th 2052.\n\nThis isn't a commercial or public release. It’s a

In [9]:
def get_collection_activity(metadata, limit, collection=None, collectionssetid=None, community=None, continuation=None):
    url = 'https://api.reservoir.tools/'
    end_point = 'collections/activity/v5'
    headers = {'accept': '*/*', 'x-api-key': f"{getenv('RESERVOIR')}"}
    if community == None:
        r = requests.get(url + end_point, params=f'?sortBy=eventTimestamp&includeMetadata={metadata}&limit={limit}', headers=headers)
    else:
        r = requests.get(url + end_point, params=f'?sortBy=eventTimestamp&includeMetadata={metadata}&limit={limit}&community={community}', headers=headers)
    data = r.json()
    return data

In [10]:
contract_db = contract_list()

test = get_collection_activity(True, 20, community='artblocks')
new_bids = []
for item in test['activities']:
    if item['type'] == 'bid':
        if item['order']['criteria']['kind'] == 'collection':
            actions = {}
            actions['contract'] = item['contract']
            actions['collection_name'] = item['collection']['collectionName']
            actions['collection_id'] = item['order']['criteria']['data']['collection']['id']
            if item['order']['criteria']['data']['collection']['id'] not in contract_db:
                new_bids.append(actions)

if new_bids:               
    contract_collection().insert_many(new_bids)

contract_db = contract_list()
df = pd.DataFrame(contract_db)

In [11]:
def get_collection_info(limit, topbid, normalize, c_id=None):
    url = 'https://api.reservoir.tools/'
    end_point = 'collections/v5'
    headers = {'accept': '*/*', 'x-api-key': f"{getenv('RESERVOIR')}"}
    r = requests.get(url + end_point, params=f'id={c_id}&includeTopBid={topbid}&normalizeRoyalties={normalize}&sortBy=allTimeVolume&limit={limit}', headers=headers)
    data = r.json()
    return data

In [20]:
new_bids = []
for c in contract_db:
    collections = get_collection_info(1, True, False, c_id=c)
    for collection in collections['collections']:
        project = {}
        project['Collection'] = collection['name']
        project['7day Volume'] = round(collection['volume']['7day'], 2)
        project['Lowest Floor Price'] = f"{str(collection['floorAsk']['sourceDomain'])} : {round(collection['floorAsk']['price']['amount']['decimal'], 3)} {collection['floorAsk']['price']['currency']['symbol']}"
        try:
            project['Highest Collection Offer'] = f"{str(collection['topBid']['sourceDomain'])} : {round(collection['topBid']['price']['amount']['decimal'], 3)} {collection['topBid']['price']['currency']['symbol']}"
            project['Spread'] = f"{round(((collection['floorAsk']['price']['amount']['decimal'] - collection['topBid']['price']['amount']['decimal']) / collection['floorAsk']['price']['amount']['decimal']) * 100, 2)}%"
        except TypeError:
            project['Highest Collection Offer'] = 'None'
            project['Spread'] = 'None'
        new_bids.append(project)
    

df = pd.DataFrame(new_bids)    

In [24]:
collections = get_collection_info(1, True, False, c_id='0x059edd72cd353df5106d2b9cc5ab83a52287ac3a:0:999999')

In [25]:
collections

{'collections': [{'id': '0x059edd72cd353df5106d2b9cc5ab83a52287ac3a:0:999999',
   'slug': 'chromie-squiggle-by-snowfro',
   'createdAt': '2022-02-12T23:31:08.732Z',
   'name': 'Chromie Squiggle by Snowfro',
   'image': 'https://media.artblocks.io/0.png',
   'banner': None,
   'discordUrl': None,
   'externalUrl': 'https://www.twitter.com/artonblockchain',
   'twitterUsername': None,
   'openseaVerificationStatus': None,
   'description': 'Simple and easily identifiable, each squiggle embodies the soul of the Art Blocks platform. Consider each my personal signature as an artist, developer, and tinkerer. Public minting of the Chromie Squiggle is permanently paused. They are now reserved for manual distribution to collectors and community members over a longer period of time. Please visit OpenSea to explore Squiggles available on the secondary market. ',
   'sampleImages': ['https://api.reservoir.tools/assets/v1?asset=fcd08d82e6950c73f7a2f92efa0d2ff7ead4cbb49594c9cb7eb02726ad6756a7be5efdf

In [13]:
collections

{'collections': [{'id': '0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270:294000000:294999999',
   'slug': 'alien-dna-by-shvembldr',
   'createdAt': '2022-04-05T17:44:25.130Z',
   'name': 'Alien DNA by Shvembldr',
   'image': 'https://media.artblocks.io/294000000.png',
   'banner': None,
   'discordUrl': None,
   'externalUrl': 'https://www.shvembldr.com',
   'twitterUsername': None,
   'openseaVerificationStatus': None,
   'description': 'In the year 3075 of the Second Nuclear Age, a laboratory of an unknown race of aliens was discovered on the planet Vtolmr while drilling a well of molecular oil. Its presumed purpose is to study genetic mutations of races from neighboring planets. During the laboratory investigation, 512 silicon impressions with DNA of different alien species were extracted. Modern methods of quantum analysis do not allow the deciphering of the data obtained. At this point, their value is purely aesthetic. But that may change in time.\n\nPress 1 to 9 - change the pixel den

In [ ]:
df[~df['Spread'].isin(['None'])].sort_values(by='Collection', ascending=True).to_csv('spread.csv', index=False)

In [ ]:
prices_db = prices_collection()

In [ ]:
pd.DataFrame(list(prices_db.find().sort('7day Volume', pymongo.DESCENDING))).drop('_id', axis=1)

In [ ]:
get_activity(False, 20, 'desc')

In [ ]:
url = 'https://api.reservoir.tools/'

response = requests.get(url, json={'limit': 20, 'includeMetadata': False}, headers=headers)

print(response.url)
test = response.json()

In [ ]:
url = "https://api.reservoir.tools/orders/bids/v5?community=artblocks&includeCriteriaMetadata=false&includeRawData=false&normalizeRoyalties=false&sortBy=createdAt&limit=50"
headers = {
    "accept": "*/*",
    "x-api-key": "demo-api-key"
}

In [ ]:
def mongo_collection():
    client = pymongo.MongoClient(getenv('MONGO'))
    db = client.nonfungible 
    return db.globalcontracts

In [ ]:
db = mongo_collection()

db.find_one()

In [ ]:
start_block = init_collection.find({'blockchain': 'flow'}).sort('blockNumber', -1).limit(1).next()['blockNumber'] - 25